In [ ]:
#import libraries
import numpy as np
import pandas as pd
import csv

In [ ]:
data = pd.read_csv("D:/Users/jklein/Downloads/CSV_Organic_Alert_Report_Aug-11-17_05-13-AM-55.csv",usecols = ['Keyword','Destination URL'], engine = 'python')

In [ ]:
#import data
data1 = pd.read_csv("D:/Users/jklein/Downloads/CSV_Organic_Alert_Report_Aug-06-17_05-13-AM-01.csv",usecols = ['Keyword','Destination URL'], engine = 'python')
data2 = pd.read_csv("D:/Users/jklein/Downloads/CSV_Organic_Alert_Report_Aug-05-17_05-31-AM-27.csv",usecols = ['Keyword','Destination URL'], engine = 'python')
data3 = pd.read_csv("D:/Users/jklein/Downloads/CSV_Organic_Alert_Report_Aug-07-17_05-16-AM-11.csv",usecols = ['Keyword','Destination URL'], engine = 'python')

In [ ]:
cumulative

In [ ]:
data['Date'] = '11/08/2017'

In [ ]:
#set date
data1['Date'] = '06/08/2017'
data2['Date'] = '05/08/2017'
data3['Date'] = '07/08/2017'

In [ ]:
#merge all data files into one dataframe
combine = data1.append([data2, data3])
data = combine.reindex(columns=['Date','Keyword','Destination URL'])
data

In [ ]:
#set the products from keywords
products = pd.read_csv("D:/Users/jklein/Documents/BSA Products/BSA Products Autodesk.csv", sep=',')
product_replacer = products.set_index('Keyword')['Product Category'].to_dict()
data['Product'] = data['Keyword']
for key in product_replacer:
    data['Product'] = data['Product'].replace(key,product_replacer.get(key))

In [ ]:
#remove duplicates
data.drop_duplicates(['Destination URL', 'Product'], inplace = True)

In [ ]:
data.shape

In [ ]:
#extract entries with duplicated URLS
dupes = data[data.duplicated('Destination URL', keep = False)]
dupes.sort_values('Destination URL', inplace = True)

In [ ]:
dupes

In [ ]:
#extract URLS with multiple product entries as a dictionary
grouped = dupes.groupby('Destination URL')['Product'].apply(list)
grouped = grouped.to_dict()
grouped

In [ ]:
dupes

In [ ]:
import string
#bring multiple products into 1 field

translator = str.maketrans('', '', '!"#$%&\'[\\]^_`{|}~')
dupes['Product2'] = dupes['Destination URL']
for key in grouped:
    dupes['Product2'] = dupes['Product2'].replace(key,str(grouped.get(key)).translate(translator))

In [ ]:
dupes['Product'] = dupes['Product2']
dupes.drop(['Product2'],axis = 1, inplace = True)
dupes

In [ ]:
dupes.drop_duplicates(['Destination URL', 'Product'], inplace = True)

In [ ]:
print(dupes.shape, data.shape)

In [ ]:
data.drop_duplicates(['Destination URL'], keep = False, inplace = True)
data = data.append(dupes)
data.shape

In [ ]:
data.to_csv("D:/Users/jklein/Documents/BSA Autodesk 08-08 pre white.csv", sep=',',
            na_rep='n/a', index=False, encoding='utf-8')

In [ ]:
whitelist = pd.read_excel("D:/Users/jklein/Downloads/whitelist.xlsx",)
whitelist

In [ ]:
sites = list(whitelist['Sites'])
sites = set(sites)
sites

In [ ]:
pattern = '|'.join(sites)
links = data[~data['Destination URL'].str.contains(pattern)]
links

In [ ]:
data

In [ ]:
links.shape

In [ ]:
links.to_csv("D:/Users/jklein/Documents/BSA Autodesk 10-08.csv", sep=',',
            na_rep='n/a', index=False, encoding='utf-8')

In [ ]:
cumulative = pd.read_excel("D:/Users/jklein/Documents/BSA Autodesk Cumulative.xlsx",na_values = 'n/a')

In [ ]:
cumulative2 = cumulative.append(links)

In [ ]:
cumulative2.drop_duplicates(['Destination URL', 'Product'], inplace = True, keep = 'first')

In [ ]:
cumulative2

In [ ]:
writer = pd.ExcelWriter("D:/Users/jklein/Documents/BSA Autodesk Cumulative.xlsx",
                        engine='xlsxwriter', datetime_format='dd/mm/yy', date_format='dd/mm/yy')
cumulative2.to_excel(writer,'Data', na_rep = 'n/a', index = False, engine='xlsxwriter')


workbook  = writer.book
worksheet1 = writer.sheets['Data']

date_format = workbook.add_format({'num_format': 'dd/mm/yy',
                                      'align': 'left'})
worksheet1.set_column('A:A', None, date_format)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
cumulative2.to_csv("D:/Users/jklein/Documents/BSA Autodesk Cumulative 11-08.csv", sep=',',
            na_rep='n/a', index=False, encoding='utf-8')